# ABS Monthly International Trade 5368

## Python set-up

In [1]:
# analytic imports
import pandas as pd
import matplotlib.pyplot as plt

# local imports
from abs_data_capture import (
    get_fs_constants,
    get_plot_constants,
    metacol,
    AbsLandingPage,
    get_abs_data,
    clear_cache,
    get_identifier,
)
from plotting import (
    clear_chart_dir,
    set_chart_dir,
    seas_trend_plot,
    recalibrate,
)

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
# in case of emergency
EMERGENCY = False
if EMERGENCY:
    clear_cache()

In [3]:
landing_page = AbsLandingPage(
    theme="economy",
    parent_topic="international-trade",
    topic="international-trade-goods-and-services-australia",
)
abs_trade = get_abs_data(landing_page)
source, CHART_DIR, Cat_ID, meta_trade = get_fs_constants(abs_trade, landing_page)
RECENT, plot_times, plot_tags = get_plot_constants(meta_trade)

About to download and cache the latest data.
Saving to cache: ABS_CACHE/36e1fd7cb4eb691e3720624f03513625--Time-series-spreadsheets-all.zip
Extracting DataFrames from the zip-file ...
Caution: Could not find the "Index" sheet in 5368092.xlsx


## Plot

### Initialisation


In [4]:
def chart_settings(chart_dir: str = CHART_DIR) -> None:
    """Set up charting parameters."""

    set_chart_dir(chart_dir)
    clear_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")


chart_settings()

### Headline charts

In [5]:
def headline_charts() -> None:
    """Produce headline charts."""

    table = "1"
    headline = meta_trade[(meta_trade[metacol.table] == table)][metacol.did].unique()

    for h in headline:
        # secure the data
        f = pd.DataFrame()
        for service_type in "Seasonally Adjusted", "Trend":
            series_id, units = get_identifier(meta_trade, h, service_type, table)
            f[service_type] = abs_trade[table][series_id]
        f_, units = recalibrate(f, units, verbose=False)

        # plot
        title = (
            h.replace("Credits", "Credits (Exports)")
            .replace("Debits", "Debits (Imports)")
            .replace(" ;", "")
            .strip()
        )
        long_line = 50
        if "," in title and len(title) > long_line:
            title = title.replace(",", ",\n")
        lfooter = "Australia. Current Prices. "
        if f_["Trend"].isna().iloc[-1]:
            last = f_["Trend"].last_valid_index()
            lfooter += f"No trend data after {last!r}. "

        seas_trend_plot(
            pd.DataFrame(f_),  # mypy
            starts=plot_times,
            tags=plot_tags,
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=lfooter,
            show=SHOW,
        )


headline_charts()

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Wed Feb 21 2024 12:38:44

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.21.0

pandas    : 2.2.0
matplotlib: 3.8.3

Watermark: 2.4.3



In [7]:
print("Finished")

Finished
